### Homework Assignment for Graduate Course in Healthcare Analytics
- Using DE-SynPUF files downloaded from https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/DE_Syn_PUF.html
- Assignment is to use methods related to decision trees to predict depression as response/target

In [13]:
# import and read file, use NaN for elements with no values
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 

df = pd.read_csv("combined_ben_sum_AddYear.csv", na_values = ['no info', '.'])
#print(df.head(5))

In [14]:
df.shape
print(df.columns)

Index(['Year', 'DESYNPUF_ID', 'BENE_BIRTH_DT', 'BENE_DEATH_DT',
       'BENE_SEX_IDENT_CD', 'BENE_RACE_CD', 'BENE_ESRD_IND', 'SP_STATE_CODE',
       'BENE_COUNTY_CD', 'BENE_HI_CVRAGE_TOT_MONS', 'BENE_SMI_CVRAGE_TOT_MONS',
       'BENE_HMO_CVRAGE_TOT_MONS', 'PLAN_CVRG_MOS_NUM', 'SP_ALZHDMTA',
       'SP_CHF', 'SP_CHRNKIDN', 'SP_CNCR', 'SP_COPD', 'SP_DEPRESSN',
       'SP_DIABETES', 'SP_ISCHMCHT', 'SP_OSTEOPRS', 'SP_RA_OA', 'SP_STRKETIA',
       'MEDREIMB_IP', 'BENRES_IP', 'PPPYMT_IP', 'MEDREIMB_OP', 'BENRES_OP',
       'PPPYMT_OP', 'MEDREIMB_CAR', 'BENRES_CAR', 'PPPYMT_CAR'],
      dtype='object')


In [42]:
#subset of data needed to answer assignment question #4
newdf = df[['DESYNPUF_ID','Year','BENE_BIRTH_DT','BENE_DEATH_DT','BENE_SEX_IDENT_CD','BENE_RACE_CD',
            'BENE_ESRD_IND','SP_STATE_CODE','BENE_COUNTY_CD','SP_ALZHDMTA',
       'SP_CHF', 'SP_CHRNKIDN', 'SP_CNCR', 'SP_COPD','SP_DEPRESSN',
       'SP_DIABETES', 'SP_ISCHMCHT', 'SP_OSTEOPRS', 'SP_RA_OA', 'SP_STRKETIA']]
print(newdf.head(5))

        DESYNPUF_ID  Year  BENE_BIRTH_DT  BENE_DEATH_DT  BENE_SEX_IDENT_CD  \
0  00013D2EFD8E45D1  2008       19230501            NaN                  1   
1  00016F745862898F  2008       19430101            NaN                  1   
2  0001FDD721E223DC  2008       19360901            NaN                  2   
3  00021CA6FF03E670  2008       19410601            NaN                  1   
4  00024B3D2352D2D0  2008       19360801            NaN                  1   

   BENE_RACE_CD BENE_ESRD_IND  SP_STATE_CODE  BENE_COUNTY_CD  SP_ALZHDMTA  \
0             1             0             26             950            2   
1             1             0             39             230            2   
2             1             0             39             280            2   
3             5             0              6             290            2   
4             1             0             52             590            2   

   SP_CHF  SP_CHRNKIDN  SP_CNCR  SP_COPD  SP_DEPRESSN  SP_DIABETES  

In [43]:
newdf.shape

(6873274, 20)

In [46]:
#calculate age (year column - year from BENE_BIRTH_DT)
#newdf["Birth_Year"] = pd.to_datetime(newdf['BENE_BIRTH_DT']).dt.year
newdf.loc[:,'Age'] = newdf.loc[:,'Year'] - (pd.to_datetime(newdf.loc[:,'BENE_BIRTH_DT'],format='%Y%m%d').dt.year)

In [47]:
#print(newdf.head(5))
display(newdf.head(5))

,DESYNPUF_ID,Year,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,SP_ALZHDMTA,...,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,Age
0,00013D2EFD8E45D1,2008,19230501,NaN,1,1,0,26,950,2,...,2,2,2,2,2,2,2,2,2,85
1,00016F745862898F,2008,19430101,NaN,1,1,0,39,230,2,...,2,2,2,2,2,2,2,2,2,65
2,0001FDD721E223DC,2008,19360901,NaN,2,1,0,39,280,2,...,2,2,2,2,2,2,2,2,2,72
3,00021CA6FF03E670,2008,19410601,NaN,1,5,0,6,290,2,...,2,2,2,2,2,2,2,2,2,67
4,00024B3D2352D2D0,2008,19360801,NaN,1,1,0,52,590,2,...,2,2,2,2,2,2,1,2,2,72


### To do
- combine patients with multiple records in a single year so we have at most one row for each patient per year
- calc and add age column
- change code values in applicable columns to 0 for no and 1 for yes
- encoding/hot encoding?
- assignment allows removal of two columns, determine which features can be removed
- try decision tree/randome forest models


In [6]:
# checking to see how many duplicate rows
dup_df = newdf[newdf.duplicated() == True].sort_values(by='DESYNPUF_ID')

In [7]:
#display(dup_df)
dup_df.shape

(1401729, 19)

In [8]:
# removing duplicate rows, 1401729 removed
df_no_duplicates = newdf.drop_duplicates(keep='first')

In [9]:
df_no_duplicates.shape

(5471545, 19)

In [11]:
#sorted_df = newdf[newdf['DESYNPUF_ID'].duplicated(keep=False)].sort_values('DESYNPUF_ID')
#print(sorted_df.head())
agg_df = df_no_duplicates.groupby('DESYNPUF_ID').filter(lambda x: len(x) > 1)


        DESYNPUF_ID  BENE_BIRTH_DT  BENE_DEATH_DT  BENE_SEX_IDENT_CD  \
0  00013D2EFD8E45D1       19230501            NaN                  1   
1  00016F745862898F       19430101            NaN                  1   
3  00021CA6FF03E670       19410601            NaN                  1   
4  00024B3D2352D2D0       19360801            NaN                  1   
5  0002DAE1C81CC70D       19431001            NaN                  1   

   BENE_RACE_CD BENE_ESRD_IND  SP_STATE_CODE  BENE_COUNTY_CD  SP_ALZHDMTA  \
0             1             0             26             950            2   
1             1             0             39             230            2   
3             5             0              6             290            2   
4             1             0             52             590            2   
5             2             0             33             400            2   

   SP_CHF  SP_CHRNKIDN  SP_CNCR  SP_COPD  SP_DEPRESSN  SP_DIABETES  \
0       2            2        2   

(4891839, 19)